In [1]:
!pip install transformers datasets peft accelerate bitsandbytes sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 17.0 MB/s eta 0:00:00


In [2]:
# Load dataset from Hugging Face
from datasets import load_dataset

dataset = load_dataset("lavita/AlpaCare-MedInstruct-52k")

# Check the dataset structure
print(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/944 [00:00<?, ?B/s]

data/train-00000-of-00001-297892d5d4e8a0(…):   0%|          | 0.00/36.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 52002
    })
})


In [3]:
def format_example(example):
    instruction = example.get("instruction", "")
    input_text = example.get("input", "")
    output_text = example.get("output", "")
    return {
        "text": f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output_text}\n\n### Disclaimer:\nThis is educational only — consult a qualified clinician."
    }

formatted_dataset = dataset.map(format_example)


Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [4]:
split_dataset = formatted_dataset["train"].train_test_split(test_size=0.1, seed=42)
val_test_split = split_dataset["test"].train_test_split(test_size=0.5, seed=42)

train_data = split_dataset["train"]
val_data = val_test_split["train"]
test_data = val_test_split["test"]


In [5]:
!pip install -U bitsandbytes transformers accelerate peft

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import torch

# Load model name
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# 8-bit quantization config (saves GPU memory)
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0
)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

print("✅ Model and tokenizer loaded successfully!")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 90.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.2
    Uninstalling transformers-4.56.2:
      Successfully uninstalled transformers-4.56.2


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ Model and tokenizer loaded successfully!


In [6]:
from peft import LoraConfig, get_peft_model

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,                      # rank
    lora_alpha=16,            # scaling factor
    target_modules=["q_proj", "v_proj"],  # layers to adapt
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA to the base model
lora_model = get_peft_model(model, lora_config)

# Print trainable parameters
lora_model.print_trainable_parameters()


trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [7]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_train = train_data.map(tokenize_function, batched=True)
tokenized_val = val_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/46801 [00:00<?, ? examples/s]

Map:   0%|          | 0/2600 [00:00<?, ? examples/s]

In [8]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("lavita/AlpaCare-MedInstruct-52k")

# Format dataset text
def format_example(example):
    instruction = example.get("instruction", "")
    input_text = example.get("input", "")
    output_text = example.get("output", "")
    return {
        "text": f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output_text}\n\n### Disclaimer:\nThis is educational only — consult a qualified clinician."
    }

formatted_dataset = dataset.map(format_example)

# Split train/val/test
split_dataset = formatted_dataset["train"].train_test_split(test_size=0.1, seed=42)
val_test_split = split_dataset["test"].train_test_split(test_size=0.5, seed=42)

train_data = split_dataset["train"]
val_data = val_test_split["train"]
test_data = val_test_split["test"]

# Tokenize
def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_train = train_data.map(tokenize_function, batched=True)
tokenized_val = val_data.map(tokenize_function, batched=True)


Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/46801 [00:00<?, ? examples/s]

Map:   0%|          | 0/2600 [00:00<?, ? examples/s]

In [9]:
def format_example(example):
    instruction = example.get("instruction", "")
    input_text = example.get("input", "")
    output_text = example.get("output", "")
    return {
        "text": f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output_text}\n\n### Disclaimer:\nThis is educational only — consult a qualified clinician."
    }

formatted_dataset = dataset.map(format_example)

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [10]:
split_dataset = formatted_dataset["train"].train_test_split(test_size=0.1, seed=42)
val_test_split = split_dataset["test"].train_test_split(test_size=0.5, seed=42)

train_data = split_dataset["train"]
val_data = val_test_split["train"]
test_data = val_test_split["test"]

In [11]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# disable cache (recommended when fine-tuning with PEFT/LoRA)
lora_model.config.use_cache = False

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir="./lora-medical-model",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    logging_steps=50,
    fp16=True,
    report_to="none",
    save_total_limit=2,   # safe to keep
    # <-- intentionally not using evaluation_strategy or save_strategy
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator
)

# Train
trainer.train()

# Run evaluation manually after training
metrics = trainer.evaluate(eval_dataset=tokenized_val)
print("Evaluation metrics:", metrics)

# Save only LoRA adapter + tokenizer (do not attempt to save the full quantized base model)
lora_model.save_pretrained("./lora_medical_adapter")
tokenizer.save_pretrained("./lora_medical_adapter")

print("✅ Training complete — LoRA adapter saved at ./lora_medical_adapter")

Step,Training Loss
50,1.203100
100,1.043800
150,1.017900
200,1.011800
250,1.000500
300,0.998700
350,0.995800
400,0.995700
450,0.993300
500,0.985400


Step,Training Loss
50,1.203100
100,1.043800
150,1.017900
200,1.011800
250,1.000500
300,0.998700
350,0.995800
400,0.995700
450,0.993300
500,0.985400


Evaluation metrics: {'eval_loss': 0.958119809627533, 'eval_runtime': 232.6076, 'eval_samples_per_second': 11.178, 'eval_steps_per_second': 2.794, 'epoch': 1.0}
✅ Training complete — LoRA adapter saved at ./lora_medical_adapter


In [16]:
from peft import PeftModel
from transformers import pipeline

# Load base model + LoRA adapter
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
instruct_model = PeftModel.from_pretrained(base_model, "./lora_medical_adapter")
instruct_model.eval()

# Reload tokenizer
tokenizer = AutoTokenizer.from_pretrained("./lora_medical_adapter")

# Create text-generation pipeline
generator = pipeline(
    "text-generation",
    model=instruct_model,
    tokenizer=tokenizer,
    max_new_tokens=200,
    temperature=0.7,
    top_p=0.9
)

# 🔍 Test prompt
prompt = """### Instruction:
Explain the importance of staying hydrated during fever.

### Input:
A 10-year-old child has mild fever and reduced appetite.

### Response:"""

output = generator(prompt)[0]['generated_text']
print(output)


Device set to use cuda:0


### Instruction:
Explain the importance of staying hydrated during fever.

### Input:
A 10-year-old child has mild fever and reduced appetite.

### Response:
Staying hydrated during a fever is crucial for preventing complications and ensuring proper health. Here are the reasons why:

1. Protects against infections: Dehydration can make it difficult for the body to fight off infections. Drinking plenty of fluids helps to replenish depleted fluids, preventing dehydration and reducing the risk of infections.

2. Promotes digestion and nutrient absorption: Drinking enough fluids helps to lubricate the stomach and improve digestion. This ensures that nutrients are properly absorbed and helps to promote overall health.

3. Helps regulate body temperature: Dehydration can lead to a higher body temperature, which can be uncomfortable and potentially dangerous. Drinking enough fluids helps to regulate body temperature, which is important for overall
